In [152]:
import re
import datetime
from datetime import date
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd

In [153]:
URL = 'https://fbref.com/en/comps/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [154]:
# Club International Cups
club_int_comps = []

for j in soup.find_all('tbody')[0].find_all("tr", {"class": "gender-m"}):
    if (j.find('td', {'data-stat':'tier'}) != None):
        if (j.find('td', {'data-stat':'tier'}).text == '1st') | (j.find('td', {'data-stat':'tier'}).text == '2nd'):
            club_int_comps.append(j.find('a')['href'])

club_int_comps

['/en/comps/14/history/Copa-Libertadores-Seasons',
 '/en/comps/205/history/Copa-Sudamericana-Seasons',
 '/en/comps/8/history/UEFA-Champions-League-Seasons',
 '/en/comps/19/history/UEFA-Europa-League-Seasons']

In [155]:
# National Team Competitions
national_comps = []

for j in soup.find_all('tbody')[1].find_all("tr", {"class": "gender-m"}):
    if (j.find('td', {'data-stat':'tier'}) != None):
        if (j.find('td', {'data-stat':'tier'}).text == '1st') | (j.find('td', {'data-stat':'tier'}).text == '2nd'):
            national_comps.append(j.find('a')['href'])

national_comps

['/en/comps/1/history/World-Cup-Seasons',
 '/en/comps/656/history/Africa-Cup-of-Nations-Seasons',
 '/en/comps/681/history/CONCACAF-Gold-Cup-Seasons',
 '/en/comps/257/history/OFC-Nations-Cup-Seasons',
 '/en/comps/676/history/European-Championship-Seasons',
 '/en/comps/664/history/AFC-Asian-Cup-Seasons']

In [156]:
# Domestic Tier 1 Comps
dom_tier1_comps = []

for j in soup.find_all('tbody')[3].find_all("tr", {"class": "gender-m"}):
    if (j.find('td') != None):
        dom_tier1_comps.append(j.find('a')['href'])

dom_tier1_comps

['/en/comps/21/history/Primera-Division-Seasons',
 '/en/comps/65/history/A-League-Seasons',
 '/en/comps/56/history/Austrian-Bundesliga-Seasons',
 '/en/comps/37/history/Belgian-First-Division-A-Seasons',
 '/en/comps/74/history/Primera-Division-Seasons',
 '/en/comps/24/history/Serie-A-Seasons',
 '/en/comps/67/history/Bulgarian-First-League-Seasons',
 '/en/comps/211/history/Canadian-Premier-League-Seasons',
 '/en/comps/35/history/Primera-Division-Seasons',
 '/en/comps/62/history/Super-League-Seasons',
 '/en/comps/41/history/Primera-A-Seasons',
 '/en/comps/63/history/1-HNL-Seasons',
 '/en/comps/66/history/Czech-First-League-Seasons',
 '/en/comps/50/history/Superliga-Seasons',
 '/en/comps/58/history/Serie-A-Seasons',
 '/en/comps/9/history/Premier-League-Seasons',
 '/en/comps/12/history/La-Liga-Seasons',
 '/en/comps/43/history/Veikkausliiga-Seasons',
 '/en/comps/13/history/Ligue-1-Seasons',
 '/en/comps/20/history/Bundesliga-Seasons',
 '/en/comps/27/history/Super-League-Greece-Seasons',
 '/en

In [157]:
all_comps = club_int_comps + national_comps + dom_tier1_comps

len(all_comps)

58

In [158]:
all_comps[-1]

'/en/comps/105/history/Liga-FUTVE-Seasons'

In [159]:
def get_all_seasons(url):
    URL = 'https://fbref.com/' + url
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    url_list = []
    
    for row in soup.find_all('tr'):
        if row.find('th',{"scope":"row"}) != None:
            url_list.append((row.find('a')['href']))
    
    return url_list

In [160]:
all_seasons = []

for i in all_comps:
    league_seasons = get_all_seasons(i)
    all_seasons += league_seasons

In [161]:
len(all_seasons)

886

In [162]:
def get_all_player_urls(url_list):
    
    final_player_urls = []

    for i in url_list:
        URL_test = 'https://fbref.com/'+ i
        page = requests.get(URL_test)
        soup = BeautifulSoup(page.content, 'html.parser')
        td1 = soup.find(lambda t: t.text.strip()=='Standard Stats')
        td2 = td1.find_next('a')
        url_to_add = td2['href']
        final_player_urls.append(url_to_add)

    return final_player_urls

In [163]:
all_standard_urls = get_all_player_urls(all_seasons)

In [164]:
len(all_standard_urls)

886

In [165]:
all_standard_urls[297]

'/en/comps/9/stats/Premier-League-Stats'

In [166]:
def get_player_table(url):
    res = requests.get('https://fbref.com/' + url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("",res.text),'lxml')
    all_tables = soup.findAll("tbody")
    player_table = all_tables[2]

    return player_table

In [148]:
# testing_table_ = get_player_table(all_standard_urls[297])

In [167]:
stats = ["player","nationality","position","squad","age","birth_year","games","games_starts","minutes","goals","assists","pens_made","pens_att","cards_yellow","cards_red","goals_per90","assists_per90","goals_assists_per90","goals_pens_per90","goals_assists_pens_per90","xg","npxg","xa","xg_per90","xa_per90","xg_xa_per90","npxg_per90","npxg_xa_per90"]

In [220]:
def get_frame(features, player_table):
    pre_df_player = dict()
    rows_player = player_table.find_all('tr')

    for row in rows_player:
        if (row.find('th',{"scope":"row"}) != None):
            for f in stats:
                cell = row.find("td",{"data-stat": f})
                a = cell.text.strip().encode()
                text=a.decode("utf-8")
                if(text == ''):
                    text = '0'
                if((f!='player')&(f!='nationality')&(f!='position')&(f!='squad')&(f!='age')&(f!='birth_year')):
                    text = float(text.replace(',',''))
                if f in pre_df_player:
                    pre_df_player[f].append(text)
                else:
                    pre_df_player[f] = [text]
    df_player = pd.DataFrame.from_dict(pre_df_player)

    return df_player

In [221]:
# test = all_standard_urls[0:3]

# lista = []

# for league in test:
#     pd.DataFrame.from_dict(get_frame(stats, get_player_table(league)))




testing_table = get_player_table(all_standard_urls[297])

get_frame(stats, testing_table)



,player,nationality,position,squad,age,birth_year,games,games_starts,minutes,goals,...,goals_pens_per90,goals_assists_pens_per90,xg,npxg,xa,xg_per90,xa_per90,xg_xa_per90,npxg_per90,npxg_xa_per90
0,Max Aarons,eng ENG,DF,Norwich City,21-302,2000,10.0,10.0,900.0,0.0,...,0.00,0.00,0.3,0.3,0.7,0.03,0.07,0.10,0.03,0.10
1,Che Adams,sct SCO,"FW,MF",Southampton,25-112,1996,7.0,6.0,482.0,1.0,...,0.19,0.37,1.3,1.3,1.2,0.24,0.22,0.47,0.24,0.47
2,Rayan Aït Nouri,fr FRA,DF,Wolves,20-149,2001,3.0,2.0,179.0,0.0,...,0.00,0.50,0.2,0.2,0.1,0.08,0.03,0.11,0.08,0.11
3,Kristoffer Ajer,no NOR,DF,Brentford,23-199,1998,7.0,6.0,565.0,0.0,...,0.00,0.00,0.2,0.2,0.3,0.03,0.05,0.07,0.03,0.07
4,Nathan Aké,nl NED,DF,Manchester City,26-257,1995,3.0,3.0,270.0,0.0,...,0.00,0.00,0.2,0.2,0.0,0.07,0.00,0.07,0.07,0.07
5,Marc Albrighton,eng ENG,"MF,FW",Leicester City,31-349,1989,3.0,2.0,191.0,1.0,...,0.47,0.47,0.0,0.0,0.2,0.02,0.10,0.12,0.02,0.12
6,Thiago Alcántara,es ESP,MF,Liverpool,30-205,1991,4.0,2.0,177.0,0.0,...,0.00,0.51,0.4,0.4,0.4,0.20,0.21,0.41,0.20,0.41
7,Trent Alexander-Arnold,eng ENG,DF,Liverpool,23-026,1998,8.0,8.0,694.0,0.0,...,0.00,0.39,0.7,0.7,3.4,0.09,0.44,0.54,0.09,0.54
8,Alisson,br BRA,GK,Liverpool,29-031,1992,9.0,9.0,810.0,0.0,...,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00
9,Allan,br BRA,MF,Everton,30-298,1991,10.0,10.0,881.0,0.0,...,0.00,0.10,0.1,0.1,0.3,0.01,0.03,0.04,0.01,0.04
